# ETL Pipeline - Last-Mile Delivery Data

Clean data pipeline for preparing features and target for ETA prediction model.


In [10]:
import pandas as pd
import numpy as np
import sys
import os

# Add parent directory to path to import from src
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))
sys.path.append('..')
sys.path.append('../src')

from etl_utils import (
    load_data, clean_data, round_timestamps_to_hour,
    prepare_features, prepare_model_data,
    calculate_correlations, plot_correlation_heatmap
)

print("ETL utilities imported successfully!")


ETL utilities imported successfully!


## 1. Load and Clean Data


In [11]:
# Load data (path relative to notebook location)
df = load_data('../Dataset/last-mile-data.csv')
print(f"Loaded: {df.shape[0]:,} records, {df.shape[1]} columns")

# Clean data
df = clean_data(df)

# Round timestamps to nearest hour
df = round_timestamps_to_hour(df)
print(f"\nAfter cleaning: {df.shape[0]:,} records")


Loaded: 72,966 records, 20 columns
Cleaned data: Removed 200 rows (0.27%)

After cleaning: 72,766 records


## 2. Feature Engineering


In [12]:
# Create all features
df = prepare_features(df)
print(f"\nFinal dataset shape: {df.shape}")


Feature Engineering:
Creating temporal features...
Creating carrier features...
  Calculating carrier features with temporal safety (no data leakage)...
Creating lane features...
  Calculating lane features with temporal safety (no data leakage)...
Creating distance features...
Creating route features...
  Calculating route features with temporal safety (no data leakage)...
Feature engineering complete. Shape: (72766, 47)

Final dataset shape: (72766, 47)


## 3. Prepare Model Data


In [13]:
# Prepare features and target
X, y, metadata = prepare_model_data(df)

print(f"\nFeatures (X): {X.shape}")
print(f"Target (y): {y.shape}")
print(f"\nFeature columns: {len(X.columns)}")
print(f"Sample rows: {len(X)}")



Model Data Preparation:
Features: 11465
Samples: 72766
Dropped columns: ['actual_transit_days', 'otd_designation', 'load_id_pseudo']

Features (X): (72766, 11465)
Target (y): (72766,)

Feature columns: 11465
Sample rows: 72766


## 4. Correlation Analysis

Note: Correlation computation is time-intensive. Set `SKIP_CORRELATIONS = False` to recompute.
The outputs below are from previous runs and remain visible for reference.


In [14]:
# Skip flag - set to False to recompute correlations (takes several minutes)
SKIP_CORRELATIONS = True

if not SKIP_CORRELATIONS:
    # Calculate correlations
    # Note: For target correlation, we need to convert datetime to numeric
    # We'll use hours from actual_ship to actual_delivery
    df_temp = df[['actual_ship', 'actual_delivery']].copy()
    df_temp['hours_until_delivery'] = (df_temp['actual_delivery'] - df_temp['actual_ship']).dt.total_seconds() / 3600
    y_numeric = df_temp['hours_until_delivery'].values

    correlations = calculate_correlations(X, pd.Series(y_numeric))

    print("Top 20 Features Correlated with Target (Hours Until Delivery):")
    print("=" * 70)
    top_features = correlations['feature_to_target'].head(20)
    for feature, corr in top_features.items():
        print(f"{feature:50s}: {corr:7.4f}")
else:
    print("Correlation computation skipped. Set SKIP_CORRELATIONS = False to recompute.")
    print("See saved outputs below from previous run.")


Correlation computation skipped. Set SKIP_CORRELATIONS = False to recompute.
See saved outputs below from previous run.


In [15]:
# Plot feature-to-target correlations
if not SKIP_CORRELATIONS:
    import matplotlib.pyplot as plt
    import os

    os.makedirs('../outputs/graphs', exist_ok=True)

    plt.figure(figsize=(10, 12))
    top_30 = correlations['feature_to_target'].head(30)
    top_30.plot(kind='barh', color='steelblue')
    plt.title('Top 30 Features Correlated with Target (Hours Until Delivery)', 
              fontsize=14, fontweight='bold')
    plt.xlabel('Correlation Coefficient')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.savefig('../outputs/graphs/feature_target_correlations.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("Skipped. See saved plot from previous run below.")


Skipped. See saved plot from previous run below.


In [16]:
# Feature-to-feature correlation heatmap (top 30 features by target correlation)
if not SKIP_CORRELATIONS:
    top_30_features = correlations['feature_to_target'].head(30).index
    corr_subset = correlations['feature_to_feature'].loc[top_30_features, top_30_features]

    fig = plot_correlation_heatmap(corr_subset, figsize=(14, 12), 
                                    title="Feature-to-Feature Correlation (Top 30 Features)")

    # Save the plot
    os.makedirs('../outputs/graphs', exist_ok=True)
    fig.savefig('../outputs/graphs/feature_feature_correlations.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("Skipped. See saved heatmap from previous run below.")


Skipped. See saved heatmap from previous run below.


## 5. Save Processed Data


In [ ]:
# Save features and target
# For target, save as hours until delivery (numeric)
df_temp = df[['actual_ship', 'actual_delivery']].copy()
df_temp['hours_until_delivery'] = (df_temp['actual_delivery'] - df_temp['actual_ship']).dt.total_seconds() / 3600
y_numeric = df_temp['hours_until_delivery'].values

# Save to outputs/data folder (path relative to notebook location)
os.makedirs('../outputs/data', exist_ok=True)

X.to_csv('../outputs/data/processed_features.csv', index=False)
pd.Series(y_numeric, name='hours_until_delivery').to_csv('../outputs/data/target.csv', index=False)

# Also save actual_ship dates for chronological splitting
df[['actual_ship']].to_csv('../outputs/data/actual_ship_dates.csv', index=False)

print("Data saved successfully!")
print(f"  - ../outputs/data/processed_features.csv: {X.shape}")
print(f"  - ../outputs/data/target.csv: {len(y_numeric)} samples")
print(f"  - ../outputs/data/actual_ship_dates.csv: {len(df)} dates (for chronological splitting)")


In [ ]:
# Also save metadata for reference
import json

metadata_to_save = {
    'feature_count': metadata['feature_count'],
    'sample_count': metadata['sample_count']
}

# Add correlations if computed
if not SKIP_CORRELATIONS and 'correlations' in dir():
    metadata_to_save['top_correlated_features'] = correlations['feature_to_target'].head(20).to_dict()

with open('../outputs/data/metadata.json', 'w') as f:
    json.dump(metadata_to_save, f, indent=2, default=str)

print("Metadata saved to ../outputs/data/metadata.json")


Metadata saved to ../outputs/data/metadata.json
